# 📦 VoxCity OBJ Export

Export voxel city models and simulation results to Wavefront OBJ format for use in external 3D software.

## Export Types

| Export | Function | Description |
|--------|----------|-------------|
| **Voxel City** | `export_obj()` | Full 3D voxel model |
| **Simulation Results** | `grid_to_obj()` | Value-mapped colored surface |

## Use Cases

- **Blender/Rhino integration** - Import for rendering and further modeling
- **Presentation** - Create high-quality visualizations
- **Analysis overlay** - View simulation results as colored 3D surfaces

## Prerequisites

```python
pip install voxcity
```

In [ ]:
# %pip install voxcity

from voxcity.generator import get_voxcity
from voxcity.exporter.obj import export_obj, grid_to_obj
from voxcity.simulator.solar import get_global_solar_irradiance_using_epw
from voxcity.simulator.view import get_view_index

meshsize = 5
rectangle_vertices = [
    (139.760, 35.680),
    (139.760, 35.690),
    (139.770, 35.690),
    (139.770, 35.680)
]

city = get_voxcity(
    rectangle_vertices,
    meshsize=meshsize,
    building_source='OpenStreetMap',
    land_cover_source='OpenStreetMap',
    canopy_height_source='High Resolution 1m Global Canopy Height Maps',
    dem_source='DeltaDTM',
    output_dir='output/obj_demo'
)

# Access grids from the VoxCity object
voxcity_grid = city.voxels.classes
dem_grid = city.dem.elevation

print(voxcity_grid.shape, dem_grid.shape)


---
## 🏙️ Export Voxel City

Export the full 3D voxel city model as an OBJ file with materials.

In [ ]:
export_obj(city, output_dir='output/obj_demo', file_name='voxcity')
print('Exported voxcity OBJ')


---
## 📊 Export Simulation Results as Colored OBJ

Export 2D analysis grids (solar, view indices) as colored 3D surfaces.

### `grid_to_obj()` Parameters

| Parameter | Description |
|-----------|-------------|
| `output_dir` | Output directory path |
| `file_name` | Output file name (without extension) |
| `cell_size` | Grid cell size (meshsize) |
| `offset` | Height offset for surface (default: view_point_height) |
| `colormap_name` | Matplotlib colormap name |
| `vmin`/`vmax` | Value range for color mapping |
| `alpha` | Transparency (0-1) |

In [ ]:
# Instantaneous solar
solar_kwargs = {
    "download_nearest_epw": True,
    "rectangle_vertices": rectangle_vertices,
    "calc_time": "01-01 12:00:00",
    "view_point_height": 1.5,
}
solar_grid = get_global_solar_irradiance_using_epw(
    city, calc_type='instantaneous', **solar_kwargs
)

# Export instantaneous solar as colored OBJ
grid_to_obj(
    solar_grid, dem_grid,
    output_dir='output/obj_demo', file_name='solar_instantaneous',
    cell_size=meshsize, offset=1.5, colormap_name='magma', num_colors=10, alpha=1.0,
    vmin=0
)

# Cumulative solar for a time window
cum_kwargs = solar_kwargs.copy()
cum_kwargs["start_time"] = "01-01 05:00:00"
cum_kwargs["end_time"] = "01-31 20:00:00"

cum_solar_grid = get_global_solar_irradiance_using_epw(
    city, calc_type='cumulative', **cum_kwargs
)

grid_to_obj(
    cum_solar_grid, dem_grid,
    output_dir='output/obj_demo', file_name='solar_cumulative',
    cell_size=meshsize, offset=1.5, colormap_name='viridis', num_colors=10, alpha=1.0
)

# View indices
gvi = get_view_index(city, mode='green', obj_export=False)

grid_to_obj(
    gvi, dem_grid,
    output_dir='output/obj_demo', file_name='gvi',
    cell_size=meshsize, offset=1.5, colormap_name='Greens', num_colors=10, alpha=1.0,
    vmin=0.0, vmax=1.0
)
